# A pyspeckit example for GBspec

This version uses pyspeckit, but you will need the "teuben-gbt" branch, where the hacking is going on to deal with missing variables in this old data.

This should reproduce Example 1 from the GBTIDL manual. The datafile you need is [here](http://safe.nrao.edu/wiki/pub/GB/Data/GBTIDLExampleAndSampleData/ngc5291.fits).

## SDFITS

We begin by disecting the typical SDFITS file, starting with raw plotting



In [ ]:
# %matplotlib inline
%matplotlib

from astropy.io import fits
from astropy import units as u
import numpy as np
from matplotlib import pyplot as plt

from pyspeckit.spectrum.readers import gbt



A useful stats routine so we can regress if we see the same data

In [ ]:
def my_stats(label,data):
    """
    for regression
    """
    mean = data.mean()
    rms  = data.std()
    dmin = data.min()
    dmax = data.max()
    print("%s  %s %s %s %s" %  (label,repr(mean),repr(rms),repr(dmin),repr(dmax)))

In [ ]:
fname = 'ngc5291.fits'
row   = 0
isrc  = 0

Start a session from a single SDFITS name. For future work we will need to be able to handle a directory (cf. the **dirin** command in GBTIDL) where several related SDFITS are located.  E.g. ARGUS uses 8.  The EDGE project has 3.

Loading a session is fast. It gives a quick overview.

In [ ]:
session = gbt.GBTSession(fname)
print(session)

In [ ]:
src = list(session.targets.keys())
if len(src) > 1:
    print("Warning: too many sources:",src)
    print("Using %s" % src[isrc])
src = src[isrc]    
print(src)

Load all blocks, as defined by a uniq (sampler,nod) pair

For GBT sampler is typically polarization (XX and YY) and multi-beam pointing (for ARGUS there are 16)

In [ ]:
session.load_target(src, verbose=True)

In [ ]:
target = session[src]
print(target)
blocks = list(target.blocks.keys())
if len(blocks) > 1:
    print("Warning: too many blocks for me to handle",blocks)


In [ ]:
nint = 0
sp0 = target.blocks[blocks[0]]
nint = len(sp0)
print("Found %d spectra in one block" % nint)

In [ ]:
sp = sp0[0]
stats = sp.stats()
print(stats)
#test  1.1189245 0.33819348 0.03492988 3.7152052
#test  1.0424037 0.3147295 0.03157423 3.3035116

In [ ]:
sp.xarr.convert_to_unit('km/s')
sp.plotter()
sp.plotter.label(verbose_label=True)   